In [1]:
import pandas as pd

import LotData.LotData as ld
import LotData.ExtractorsAndTables as ent
import utility.LoggingUtility as lut
import utility.webscrapingUtil as wut
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import CW_Scraper
import sqlalchemy
import time
import numpy as np
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from Runnables.RunningSettings import wanted_categories
import database.DatabaseManager as dbm

In [7]:
not_final_query = text("select bid.lid,meta_timestamp,meta.category_int,meta.category_name from bid INNER JOIN (select max(amount) as max_amount,bid.lid from bid group by lid) as mb on mb.lid = bid.lid AND mb.max_amount = bid.amount INNER JOIN meta ON meta.lid = bid.lid WHERE is_final_bid is false ORDER BY meta_timestamp asc;")

session,engine = dbm.getSessionEngine()

lids = session.execute(not_final_query).fetchall()
db_manager = dbm.DatabaseManager(session,engine)

In [8]:
err_lids = []
for (LID,meta_timestamp,category_int,category_name) in lids:
        t1 = time.time()
        meta_data = ent.MetadataExtractor(LID,meta_timestamp,category_int,category_name)

        l_data = ld.LotData(meta_data)
        for record_key in l_data.keys():
            table = wut.recordIntoTabe(record_key)
            query = f"SELECT EXISTS (SELECT 1 FROM {table} WHERE lid = :lid)"

            result = session.execute(text(query), {'lid': LID})
            exists = result.scalar()  # or fetch the result as required
            if(table not in ["spec","auction"]):
                exists = False
            if(not exists):
                try:

                    df = l_data[record_key]
                    if(table == "bid" and LID == 85264349):
                        print(df)
                    if(record_key == "meta_record"):
                        df["status"] = "idle"
                    db_manager.insertRecordDataframe(df,wut.recordIntoTabe(record_key))
                except Exception as e:
                    if(LID == 85264349 and table == "bid"):
                        print(e)
                    err_lids += [(LID,e)]

        time.sleep(np.random.uniform(0,0.01))



non_expected_errors = [err for err in err_lids if len(err) > 1 and (type(err[1]) != sqlalchemy.exc.IntegrityError) and (type(err[1]) != KeyError)]